In [1]:
# Import for preprocessing

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline as make_imb_pipeline
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection  import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression


from sklearn import set_config
set_config(transform_output='pandas')

In [2]:
# import

df = pd.read_csv('merged_clean.csv')
df

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles
0,2009.0,PHI,ACT,70.0,200.0,00-0000108,12.0,WC,1997.0,35.0,David Akers,K,00-0000108_2009_David_Akers_K,18.0,18.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010.0,PHI,ACT,70.0,200.0,00-0000108,13.0,WC,1997.0,36.0,David Akers,K,00-0000108_2010_David_Akers_K,18.0,18.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hamstring,Active,Hamstring,Full Participation in Practice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2011.0,SF,ACT,70.0,200.0,00-0000108,14.0,CON,1997.0,37.0,David Akers,K,00-0000108_2011_David_Akers_K,20.0,20.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.047313,1.0,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2012.0,SF,ACT,70.0,200.0,00-0000108,15.0,SB,1997.0,38.0,David Akers,K,00-0000108_2012_David_Akers_K,21.0,21.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Hip,Probable,Hip,Full Participation in Practice,0.0,0.0,5.0,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2013.0,DET,ACT,70.0,200.0,00-0000108,16.0,REG,1997.0,39.0,David Akers,K,00-0000108_2013_David_Akers_K,17.0,17.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13494,2024.0,SEA,CUT,67.0,195.0,00-0039940,0.0,REG,2024.0,24.0,Kairee Robinson,RB,00-0039940_2024_Kairee_Robinson_RB,0.0,18.0,18.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13495,2024.0,KC,CUT,77.0,240.0,00-0039941,0.0,REG,2024.0,25.0,Geor'Quarius Spivey,TE,00-0039941_2024_Geor'Quarius_Spivey_TE,0.0,22.0,22.0,1999.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [3]:
df = df.sort_values(['player_id_x', 'season_x'], ascending=False)
df

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles
13498,2024.0,SEA,CUT,78.0,240.0,00-0039952,0.0,REG,2024.0,25.0,Devon Garrison,TE,00-0039952_2024_Devon_Garrison_TE,0.0,18.0,18.0,1999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13497,2024.0,GB,DEV,72.0,205.0,00-0039945,0.0,WC,2024.0,26.0,Alex Hale,K,00-0039945_2024_Alex_Hale_K,0.0,19.0,19.0,1998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13496,2024.0,NO,CUT,76.0,260.0,00-0039942,0.0,REG,2024.0,23.0,Mason Fairchild,TE,00-0039942_2024_Mason_Fairchild_TE,0.0,18.0,18.0,2001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13495,2024.0,KC,CUT,77.0,240.0,00-0039941,0.0,REG,2024.0,25.0,Geor'Quarius Spivey,TE,00-0039941_2024_Geor'Quarius_Spivey_TE,0.0,22.0,22.0,1999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13494,2024.0,SEA,CUT,67.0,195.0,00-0039940,0.0,REG,2024.0,24.0,Kairee Robinson,RB,00-0039940_2024_Kairee_Robinson_RB,0.0,18.0,18.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2013.0,DET,ACT,70.0,200.0,00-0000108,16.0,REG,1997.0,39.0,David Akers,K,00-0000108_2013_David_Akers_K,17.0,17.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2012.0,SF,ACT,70.0,200.0,00-0000108,15.0,SB,1997.0,38.0,David Akers,K,00-0000108_2012_David_Akers_K,21.0,21.0,0.0,1974.0,0.0,

In [4]:
df['years_exp']

# make sure you are cycling over players

#filter and shift for only each play

# drop nans

df['next_season_missed'] = df.groupby('player_id_x')['missed_games'].shift(1)

In [5]:
df

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,next_season_missed
13498,2024.0,SEA,CUT,78.0,240.0,00-0039952,0.0,REG,2024.0,25.0,Devon Garrison,TE,00-0039952_2024_Devon_Garrison_TE,0.0,18.0,18.0,1999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13497,2024.0,GB,DEV,72.0,205.0,00-0039945,0.0,WC,2024.0,26.0,Alex Hale,K,00-0039945_2024_Alex_Hale_K,0.0,19.0,19.0,1998.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13496,2024.0,NO,CUT,76.0,260.0,00-0039942,0.0,REG,2024.0,23.0,Mason Fairchild,TE,00-0039942_2024_Mason_Fairchild_TE,0.0,18.0,18.0,2001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13495,2024.0,KC,CUT,77.0,240.0,00-0039941,0.0,REG,2024.0,25.0,Geor'Quarius Spivey,TE,00-0039941_2024_Geor'Quarius_Spivey_TE,0.0,22.0,22.0,1999.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13494,2024.0,SEA,CUT,67.0,195.0,00-0039940,0.0,REG,2024.0,24.0,Kairee Robinson,RB,00-0039940_2024_Kairee_Robinson_RB,0.0,18.0,18.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2013.0,DET,ACT,70.0,200.0,00-0000108,16.0,REG,1997.0,39.0,David Akers,K,00-0000108_2013_David_Akers_K,17.0,17.0,0.0,1974.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,2012.0,SF,ACT,70.0,200.0,00-0000108,15.0,SB,1997.0,38.0,David Akers,K,00-00001

In [6]:
df['next_season_missed'].isna().sum()

3438

In [7]:
df['player_id_x'].nunique()

3438

In [8]:
# df = df.dropna(subset='next_season_missed')

In [9]:
df[(df['years_exp'] > 0) & (df['season_x'] == 2024)]

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,next_season_missed
13272,2024.0,DET,ACT,70.0,202.0,00-0039172,1.0,DIV,2023.0,25.0,Jake Bates,K,00-0039172_2024_Jake_Bates_K,18.0,20.0,2.0,1999.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13271,2024.0,SEA,ACT,73.0,220.0,00-0039165,1.0,REG,2023.0,23.0,Zach Charbonnet,RB,00-0039165_2024_Zach_Charbonnet_RB,17.0,18.0,1.0,2001.0,0.087690,-0.013126,0.157678,0.070336,0.047619,0.033175,0.034483,0.519393,0.125345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,135.0,569.0,8.0,0.0,0.0,32.0,0.0,42.0,52.0,308.0,1.0,0.0,0.0,-54.0,345.0,7.0,0.0,0.0,Elbow,Active,Elbow,Full Participation in Practice,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13269,2024.0,IND,INA,76.0,232.0,00-0039164,1.0,REG,2023.0,22.0,Anthony Richardson,QB,00-0039164_2024_Anthony_Richardson_QB,13.0,18.0,5.0,2002.0,0.003448,-0.002288,0.008235,-0.000505,0.000000,0.000000,0.000000,-0.003448,0.218343,126.0,264.0,1814.0,8.0,12.0,14.0,115.0,3.0,2.0,3224.0,73.0,0.0,86.0,499.0,6.0,6.0,1.0,33.0,1.0,1.0,1.0,-1.0,0.0,0.0,0.0,-8.0,7.0,0.0,0.0,0.0,Back,Out,Back,Did Not Participate In Practice,0.0,0.0,0.0,3.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,NaN
13267,2024.0,HOU,ACT,75.0,218.0,00-0039163,1.0,DIV,2023.0,23.0,C.J. Stroud,QB,00-0039163_2024_C.J._Stroud_QB,19.0,20.0,1.0,2001.0,0.001585,0.001908,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.165953,377.0,593.0,4254.0,21.0,13.0,63.0,487.0,5.0,2.0,5011.0,211.0,0.0,64.0,317.0,0.0,3.0,2.0,19.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,Hand,Active,Hand,Full Participation in Practice,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
13265,2024.0,TEN,ACT,76.0,232.0,00-0039152,1.0,REG,2023.0,25.0,Will Levis,QB,00-0039152_2024_Will_Levis_QB,14.0,18.0,4.0,1999.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.140709,190.0,301.0,2091.0,13.0,12.0,41.0,233.0,5.0,3.0,2621.0,86.0,1.0,45.0,183.0,0.0,5.0,3.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Shoulder,Questionable,Shoulder,Limited Participation in Practice,0.0,0.0,0.0,4.0,3.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,2024.0,ATL,RET,76.0,217.0,00-0026143,16.0,REG,2008.0,39.0,Matt Ryan,QB,00-0026143_2024_Matt_Ryan_QB,0.0,18.0,18.0,1985.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.

In [10]:
train_df = df[df['season_x'] < 2023]
test_df = df[df['season_x'] == 2023]

In [11]:
test_df

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,next_season_missed
13270,2023.0,SEA,ACT,73.0,220.0,00-0039165,0.0,REG,2023.0,22.0,Zach Charbonnet,RB,00-0039165_2023_Zach_Charbonnet_RB,16.0,18.0,2.0,2001.0,0.072595,-0.010534,0.125506,0.052938,0.000,0.039326,0.035176,0.379310,0.080979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.0,462.0,1.0,0.0,0.0,32.0,0.0,33.0,40.0,209.0,0.0,0.0,0.0,-41.0,248.0,7.0,0.0,0.0,Knee,Questionable,Knee,Full Participation in Practice,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13268,2023.0,IND,RES,76.0,232.0,00-0039164,0.0,REG,2023.0,21.0,Anthony Richardson,QB,00-0039164_2023_Anthony_Richardson_QB,4.0,18.0,14.0,2002.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.213689,50.0,84.0,577.0,3.0,1.0,7.0,29.0,1.0,0.0,676.0,31.0,2.0,25.0,136.0,4.0,2.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Concussion,Active,Concussion,Full Participation in Practice,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0
13266,2023.0,HOU,ACT,75.0,218.0,00-0039163,0.0,DIV,2023.0,22.0,C.J. Stroud,QB,00-0039163_2023_C.J._Stroud_QB,17.0,20.0,3.0,2001.0,0.001779,-0.000201,0.000491,0.000000,0.000,0.000000,0.000000,0.000000,0.206308,354.0,553.0,4557.0,26.0,5.0,38.0,331.0,6.0,3.0,4908.0,206.0,0.0,43.0,167.0,3.0,2.0,1.0,17.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,Concussion,Active,Concussion,Full Participation in Practice,1.0,0.0,0.0,1.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0
13264,2023.0,TEN,INA,76.0,232.0,00-0039152,0.0,REG,2023.0,24.0,Will Levis,QB,00-0039152_2023_Will_Levis_QB,9.0,18.0,9.0,1999.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.159821,149.0,255.0,1808.0,8.0,4.0,28.0,185.0,5.0,2.0,2688.0,81.0,1.0,25.0,57.0,1.0,2.0,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Foot,Questionable,Foot,Limited Participation in Practice,0.0,0.0,0.0,2.0,8.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0
13262,2023.0,CAR,ACT,70.0,194.0,00-0039150,0.0,REG,2023.0,22.0,Bryce Young,QB,00-0039150_2023_Bryce_Young_QB,16.0,18.0,2.0,2001.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.154486,315.0,527.0,2877.0,11.0,10.0,62.0,477.0,9.0,6.0,4009.0,133.0,1.0,39.0,253.0,0.0,2.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Back,Active,Back,Full Participation in Practice,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1785,2023.0,NYG,ACT,73.0,207.0,00-0025580,16.0,REG,2007.0,39.0,Mason Crosby,K,00-0025580_2023_Mason_Crosby_K,3.0,18.0,15.0,1984.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000

In [12]:
test_df['next_season_missed'].isna().sum()

173

In [13]:
test_df = test_df.dropna(subset='next_season_missed')

In [14]:
test_df['season_x'].value_counts()

2023.0    793
Name: season_x, dtype: int64

In [15]:
test_df

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,next_season_missed
13270,2023.0,SEA,ACT,73.0,220.0,00-0039165,0.0,REG,2023.0,22.0,Zach Charbonnet,RB,00-0039165_2023_Zach_Charbonnet_RB,16.0,18.0,2.0,2001.0,0.072595,-0.010534,0.125506,0.052938,0.000,0.039326,0.035176,0.379310,0.080979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,108.0,462.0,1.0,0.0,0.0,32.0,0.0,33.0,40.0,209.0,0.0,0.0,0.0,-41.0,248.0,7.0,0.0,0.0,Knee,Questionable,Knee,Full Participation in Practice,0.0,0.0,0.0,2.0,2.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
13268,2023.0,IND,RES,76.0,232.0,00-0039164,0.0,REG,2023.0,21.0,Anthony Richardson,QB,00-0039164_2023_Anthony_Richardson_QB,4.0,18.0,14.0,2002.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.213689,50.0,84.0,577.0,3.0,1.0,7.0,29.0,1.0,0.0,676.0,31.0,2.0,25.0,136.0,4.0,2.0,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Concussion,Active,Concussion,Full Participation in Practice,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,5.0
13266,2023.0,HOU,ACT,75.0,218.0,00-0039163,0.0,DIV,2023.0,22.0,C.J. Stroud,QB,00-0039163_2023_C.J._Stroud_QB,17.0,20.0,3.0,2001.0,0.001779,-0.000201,0.000491,0.000000,0.000,0.000000,0.000000,0.000000,0.206308,354.0,553.0,4557.0,26.0,5.0,38.0,331.0,6.0,3.0,4908.0,206.0,0.0,43.0,167.0,3.0,2.0,1.0,17.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,Concussion,Active,Concussion,Full Participation in Practice,1.0,0.0,0.0,1.0,3.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,1.0
13264,2023.0,TEN,INA,76.0,232.0,00-0039152,0.0,REG,2023.0,24.0,Will Levis,QB,00-0039152_2023_Will_Levis_QB,9.0,18.0,9.0,1999.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.159821,149.0,255.0,1808.0,8.0,4.0,28.0,185.0,5.0,2.0,2688.0,81.0,1.0,25.0,57.0,1.0,2.0,2.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Foot,Questionable,Foot,Limited Participation in Practice,0.0,0.0,0.0,2.0,8.0,0.0,2.0,0.0,0.0,6.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0
13262,2023.0,CAR,ACT,70.0,194.0,00-0039150,0.0,REG,2023.0,22.0,Bryce Young,QB,00-0039150_2023_Bryce_Young_QB,16.0,18.0,2.0,2001.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.154486,315.0,527.0,2877.0,11.0,10.0,62.0,477.0,9.0,6.0,4009.0,133.0,1.0,39.0,253.0,0.0,2.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Back,Active,Back,Full Participation in Practice,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2028,2023.0,CLE,ACT,78.0,245.0,00-0026158,15.0,WC,2008.0,38.0,Joe Flacco,QB,00-0026158_2023_Joe_Flacco_QB,7.0,19.0,12.0,1985.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.184174,15

In [16]:
train_df['next_season_missed'].isna().sum()

2229

In [17]:
train_df = train_df.dropna(subset='next_season_missed')

In [18]:
train_df

,season_x,team_x,status,height,weight,player_id_x,years_exp,game_type_x,entry_year,age,cleaned_name_x,std_pos_x,unique_id,active_games,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,report_primary_injury,report_status,practice_primary_injury,practice_status,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,next_season_missed
12910,2022.0,ATL,ACT,72.0,193.0,00-0038149,0.0,REG,2022.0,23.0,Josh Ali,WR,00-0038149_2022_Josh_Ali_WR,2.0,18.0,16.0,1999.0,0.033333,0.127119,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,45.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0
12907,2022.0,CLE,DEV,74.0,219.0,00-0038147,0.0,REG,2022.0,23.0,Daylen Baldwin,WR,00-0038147_2022_Daylen_Baldwin_WR,1.0,18.0,17.0,1999.0,0.071429,0.193798,0.00000,0.155280,0.0,0.200000,0.181818,0.892857,0.069402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,25.0,0.0,0.0,0.0,25.0,0.0,2.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
12904,2022.0,TEN,DEV,78.0,260.0,00-0038145,0.0,REG,2022.0,25.0,Thomas Odukoya,TE,00-0038145_2022_Thomas_Odukoya_TE,0.0,18.0,18.0,1997.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
12900,2022.0,SEA,ACT,69.0,210.0,00-0038134,0.0,WC,2022.0,22.0,Kenneth Walker,RB,00-0038134_2022_Kenneth_Walker_RB,16.0,19.0,3.0,2000.0,0.066176,-0.018262,0.13046,0.041217,0.0,0.030303,0.026667,0.308824,0.147303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,243.0,1113.0,10.0,0.0,0.0,55.0,0.0,28.0,36.0,168.0,0.0,0.0,0.0,-75.0,224.0,6.0,0.0,0.0,Ankle,Active,Ankle,Limited Participation in Practice,0.0,0.0,0.0,5.0,6.0,3.0,3.0,0.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
12897,2022.0,CAR,RES,74.0,205.0,00-0038132,0.0,REG,2022.0,23.0,Matt Corral,QB,00-0038132_2022_Matt_Corral_QB,0.0,18.0,18.0,1999.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,No Inj,Active,No Inj,Full Participation In Practice,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2009.0,PIT,ACT,74.0,216.0,00-0000865,11.0,REG,1998.0,35.0,Charlie Batch,QB,00-0000865_2009_Charlie_Batch_QB,17.0,17.0,0.0,1974.0,0.000000,0.000000,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.005375,1.0,2.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [19]:
col_list = train_df.columns.to_list()

In [20]:
col_list

['season_x',
 'team_x',
 'status',
 'height',
 'weight',
 'player_id_x',
 'years_exp',
 'game_type_x',
 'entry_year',
 'age',
 'cleaned_name_x',
 'std_pos_x',
 'unique_id',
 'active_games',
 'max_week',
 'missed_games',
 'birth_year',
 'tgt_sh',
 'ay_sh',
 'yac_sh',
 'ry_sh',
 'rtd_sh',
 'rfd_sh',
 'rtdfd_sh',
 'yptmpa',
 'ppr_sh',
 'completions_summed',
 'attempts_summed',
 'passing_yards_summed',
 'passing_tds_summed',
 'interceptions_summed',
 'sacks_summed',
 'sack_yards_summed',
 'sack_fumbles_summed',
 'sack_fumbles_lost_summed',
 'passing_air_yards_summed',
 'passing_first_downs_summed',
 'passing_2pt_conversions_summed',
 'carries_summed',
 'rushing_yards_summed',
 'rushing_tds_summed',
 'rushing_fumbles_summed',
 'rushing_fumbles_lost_summed',
 'rushing_first_downs_summed',
 'rushing_2pt_conversions_summed',
 'receptions_summed',
 'targets_summed',
 'receiving_yards_summed',
 'receiving_tds_summed',
 'receiving_fumbles_summed',
 'receiving_fumbles_lost_summed',
 'receiving_air

In [21]:
X_train = train_df.drop(columns=['status', 'player_id_x', 'cleaned_name_x', 'unique_id', 'active_games', 'report_primary_injury',
                                 'report_status', 'practice_primary_injury', 'practice_status', 'next_season_missed'])

y_train = train_df['next_season_missed'].copy()

X_test = test_df.drop(columns=['status', 'player_id_x', 'cleaned_name_x', 'unique_id', 'active_games', 'report_primary_injury',
                                 'report_status', 'practice_primary_injury', 'practice_status', 'next_season_missed'])

y_test = test_df['next_season_missed'].copy()

In [22]:
X_train.columns

Index(['season_x', 'team_x', 'height', 'weight', 'years_exp', 'game_type_x',
       'entry_year', 'age', 'std_pos_x', 'max_week', 'missed_games',
       'birth_year', 'tgt_sh', 'ay_sh', 'yac_sh', 'ry_sh', 'rtd_sh', 'rfd_sh',
       'rtdfd_sh', 'yptmpa', 'ppr_sh', 'completions_summed', 'attempts_summed',
       'passing_yards_summed', 'passing_tds_summed', 'interceptions_summed',
       'sacks_summed', 'sack_yards_summed', 'sack_fumbles_summed',
       'sack_fumbles_lost_summed', 'passing_air_yards_summed',
       'passing_first_downs_summed', 'passing_2pt_conversions_summed',
       'carries_summed', 'rushing_yards_summed', 'rushing_tds_summed',
       'rushing_fumbles_summed', 'rushing_fumbles_lost_summed',
       'rushing_first_downs_summed', 'rushing_2pt_conversions_summed',
       'receptions_summed', 'targets_summed', 'receiving_yards_summed',
       'receiving_tds_summed', 'receiving_fumbles_summed',
       'receiving_fumbles_lost_summed', 'receiving_air_yards_summed',
       're

In [23]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9268 entries, 12910 to 0
Data columns (total 67 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   season_x                            9268 non-null   float64
 1   team_x                              9268 non-null   object 
 2   height                              9268 non-null   float64
 3   weight                              9268 non-null   float64
 4   years_exp                           9268 non-null   float64
 5   game_type_x                         9268 non-null   object 
 6   entry_year                          9268 non-null   float64
 7   age                                 9268 non-null   float64
 8   std_pos_x                           9268 non-null   object 
 9   max_week                            9268 non-null   float64
 10  missed_games                        9268 non-null   float64
 11  birth_year                          9268 n

In [24]:
# Make transformers for preprocess

ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
scaler = MinMaxScaler()

In [25]:
ohe_cols = list(X_train.select_dtypes('object').columns)
nums_cols = list(X_train.select_dtypes('float64').columns)

In [26]:
# Make the encoder for pipeline

proc_encode = ColumnTransformer([
  ('ohe', ohe, ohe_cols)
], remainder = 'passthrough', verbose_feature_names_out=False)

In [27]:
# Scaler transformer for pipeline

proc_scale = ColumnTransformer([
  ('scaler', scaler, nums_cols)
], remainder='passthrough', verbose_feature_names_out=False)

In [28]:
main_pipe = Pipeline([
  ('proc_encode', proc_encode),
  ('proc_scale', proc_scale)
])

In [29]:
main_pipe.fit(X_train)

c:\Users\nick_\anaconda3\envs\foozball\lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('proc_encode',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['team_x', 'game_type_x',
                                                   'std_pos_x'])],
                                   verbose_feature_names_out=False)),
                ('proc_scale',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', MinMaxScaler(),
                                                  ['season_x', 'height',
                                                   'weight'...
                                                   'rtdfd_sh', 'yptmpa',
                                                   'ppr_sh',
                                                   'completions_summed',
                                                   'attempts_summed',
                                                   'passing_yards_summed',
                                                   'passing_tds_summed',
                                                   'interceptions_summed',
                                                   'sacks_summed',
                                                   'sack_yards_summed',
                                                   'sack_fumbles_summed',
                                                   'sack_fumbles_lost_summed',
                                                   'passing_air_yards_summed',
                                                   'passing_first_downs_summed',
                                                   'passing_2pt_conversions_summed', ...])],
                                   verbose_feature_names_out=False))])

In [30]:
X_train_proc = main_pipe.transform(X_train)
X_test_proc = main_pipe.transform(X_test)

In [31]:
X_train_proc.head()

,season_x,height,weight,years_exp,entry_year,age,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,team_x_ARI,team_x_ARZ,team_x_ATL,team_x_BAL,team_x_BLT,team_x_BUF,team_x_CAR,team_x_CHI,team_x_CIN,team_x_CLE,team_x_CLV,team_x_DAL,team_x_DEN,team_x_DET,team_x_GB,team_x_HOU,team_x_HST,team_x_IND,team_x_JAX,team_x_KC,team_x_LA,team_x_LAC,team_x_LV,team_x_MIA,team_x_MIN,team_x_NE,team_x_NO,team_x_NYG,team_x_NYJ,team_x_OAK,team_x_PHI,team_x_PIT,team_x_SD,team_x_SEA,team_x_SF,team_x_SL,team_x_TB,team_x_TEN,team_x_WAS,game_type_x_CON,game_type_x_DIV,game_type_x_REG,game_type_x_SB,game_type_x_WC,std_pos_x_K,std_pos_x_QB,std_pos_x_RB,std_pos_x_TE,std_pos_x_WR
12910,1.0,0.4375,0.267081,0.0,1.0,0.076923,0.2,0.727273,0.945946,0.083333,0.350538,0.072107,0.057877,0.0,0.000000,0.000000,0.040199,0.478312,0.0,0.0,0.000928,0.0,0.0,0.0,0.0,0.0,0.0,0.001681,0.0,0.0,0.000000,0.014805,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.004292,0.006555,0.0,0.0,0.0,0.058069,0.007547,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12907,1.0,0.5625,0.428571,0.0,1.0,0.076923,0.2,0.772727,0.945946,0.178571,0.457429,0.072107,0.377819,0.0,0.434667,0.389006,0.205302,0.583631,0.0,0.0,0.000928,0.0,0.0,0.0,0.0,0.0,0.0,0.001681,0.0,0.0,0.000000,0.014805,0.00000,0.0,0.0,0.000000,0.0,0.011236,0.008584,0.016796,0.0,0.0,0.0,0.051317,0.007547,0.018519,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12904,1.0,0.8125,0.683230,0.0,1.0,0.153846,0.2,0.818182,0.891892,0.000000,0.146759,0.072107,0.057877,0.0,0.000000,0.000000,0.040199,0.478312,0.0,0.0,0.000928,0.0,0.0,0.0,0.0,0.0,0.0,0.001681,0.0,0.0,0.000000,0.014805,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.006555,0.0,0.0,0.0,0.042876,0.007547,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12900,1.0,0.2500,0.372671,0.0,1.0,0.038462,0.4,0.136364,0.972973,0.165441,0.117485,0.316693,0.142801,0.0,0.065859,0.057054,0.097305,0.701848,0.0,0.0,0.000928,0.0,0.0,0.0,0.0,0.0,0.0,0.001681,0.0,0.0,0.557339,0.514132,0.47619,0.0,0.0,0.514019,0.0,0.157303,0.154506,0.075379,0.0,0.0,0.0,0.017556,0.218868,0.055556,0.0,0.0,0.0,0.0,0.0,0.333333,0.315789,0.176471,0.157895,0.0,0.066667,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
12897,1.0,0.5625,0.341615,0.0,1.0,0.076923,0.2,0.818182,0.945946,0.000000,0.14675

In [32]:
lr = LinearRegression()

lr.fit(X_train_proc, y_train)

LinearRegression()

In [33]:
X_test_proc = main_pipe.transform(X_test)

In [34]:
X_test_proc.head()

,season_x,height,weight,years_exp,entry_year,age,max_week,missed_games,birth_year,tgt_sh,ay_sh,yac_sh,ry_sh,rtd_sh,rfd_sh,rtdfd_sh,yptmpa,ppr_sh,completions_summed,attempts_summed,passing_yards_summed,passing_tds_summed,interceptions_summed,sacks_summed,sack_yards_summed,sack_fumbles_summed,sack_fumbles_lost_summed,passing_air_yards_summed,passing_first_downs_summed,passing_2pt_conversions_summed,carries_summed,rushing_yards_summed,rushing_tds_summed,rushing_fumbles_summed,rushing_fumbles_lost_summed,rushing_first_downs_summed,rushing_2pt_conversions_summed,receptions_summed,targets_summed,receiving_yards_summed,receiving_tds_summed,receiving_fumbles_summed,receiving_fumbles_lost_summed,receiving_air_yards_summed,receiving_yards_after_catch_summed,receiving_first_downs_summed,receiving_2pt_conversions_summed,special_teams_tds_summed,doubt,out,prob,quest,fp,dnp,lp,Hamstring,Knee,Ankle,Groin,Neck,Concussion,Foot,Back,Achilles,team_x_ARI,team_x_ARZ,team_x_ATL,team_x_BAL,team_x_BLT,team_x_BUF,team_x_CAR,team_x_CHI,team_x_CIN,team_x_CLE,team_x_CLV,team_x_DAL,team_x_DEN,team_x_DET,team_x_GB,team_x_HOU,team_x_HST,team_x_IND,team_x_JAX,team_x_KC,team_x_LA,team_x_LAC,team_x_LV,team_x_MIA,team_x_MIN,team_x_NE,team_x_NO,team_x_NYG,team_x_NYJ,team_x_OAK,team_x_PHI,team_x_PIT,team_x_SD,team_x_SEA,team_x_SF,team_x_SL,team_x_TB,team_x_TEN,team_x_WAS,game_type_x_CON,game_type_x_DIV,game_type_x_REG,game_type_x_SB,game_type_x_WC,std_pos_x_K,std_pos_x_QB,std_pos_x_RB,std_pos_x_TE,std_pos_x_WR
13270,1.076923,0.5000,0.434783,0.0,1.028571,0.038462,0.2,0.090909,1.000000,0.181488,0.129872,0.307405,0.166952,0.0,0.085468,0.07526,0.110340,0.601199,0.000000,0.000000,0.000928,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.001681,0.000000,0.000,0.247706,0.222073,0.047619,0.000000,0.000000,0.299065,0.000000,0.185393,0.171674,0.092175,0.0,0.0,0.0,0.029034,0.241509,0.064815,0.0,0.0,0.000000,0.0,0.0,0.133333,0.105263,0.058824,0.000000,0.117647,0.066667,0.000000,0.0,0.0,0.000000,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13268,1.076923,0.6875,0.509317,0.0,1.028571,0.000000,0.2,0.636364,1.027027,0.000000,0.146759,0.072107,0.057877,0.0,0.000000,0.00000,0.040199,0.802590,0.091912,0.103704,0.090122,0.050000,0.033333,0.100000,0.057199,0.076923,0.000,0.096372,0.092814,0.250,0.057339,0.075819,0.190476,0.181818,0.166667,0.074766,0.000000,0.000000,0.000000,0.006555,0.0,0.0,0.0,0.042876,0.007547,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.052632,0.058824,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.222222,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13266,1.076923,0.6250,0.422360,0.0,1.028571,0.038462,0.6,0.136364,1.000000,0.004448,0.146437,0.073028,0.057877,0.0,0.000000,0.00000,0.040199,0.791390,0.650735,0.682716,0.705364,0.433333,0.166667,0.542857,0.652860,0.461538,0.375,0.689172,0.616766,0.000,0.098624,0.089726,0.142857,0.181818,0.166667,0.158879,0.333333,0.005618,0.004292,0.006555,0.0,0.0,0.0,0.042539,0.008491,0.000000,0.0,0.0,0.166667,0.0,0.0,0.066667,0.157895,0.117647,0.052632,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.0000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13264,1.076923,0.6875,0.509317,0.0,1.028571,0.115385,0.2,0.409091,0.945946,0.000000,0.146759,0.072107,0.057877,0.0,0.000000,0.00000,0.040199,0.720844,0.273897,0.314815,0.280414,0.133333,0.133333,0.400000,0.364892,0.384615,0.250,0.378204,0.242515,0.125,0.057339,0.040377,0.047619,0.181818,0.333333,0.065421,0.000000,0.000000,0.000000,0.006555,0.0,0.0,0.0,0.042876,0.007547,0.000000,0.0,0.0,0.0000

In [35]:
# Functions for metrics to evaluate regression

from sklearn.metrics import (mean_absolute_error, mean_squared_error, root_mean_squared_error,
r2_score, mean_absolute_percentage_error)

def adj_r2(r2, x):
  n = x.shape[0]
  p = x.shape[1]
  return 1 - (((n-1) / (n - p- 1)) * (1 - r2))

def evaluate_regression(model, X, y, name='model'):
  y_pred = model.predict(X)
  MAPE = mean_absolute_percentage_error(y, y_pred)
  r2 = r2_score(y, y_pred)
  RMSE = root_mean_squared_error(y, y_pred)
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  a_r2 = adj_r2(r2, X)
  metrics = ['MAE','MSE','RMSE','MAPE','R2','adj_r2']
  results = pd.DataFrame(columns=metrics, index=[name])
  results['MAE'] = [MAE]
  results['MSE'] = [MSE]
  results['RMSE'] = RMSE
  results['MAPE'] = MAPE
  results['R2'] = r2
  results['adj_r2'] = a_r2

  return results

In [36]:
first_results = evaluate_regression(lr, X_test_proc, y_test, 'lin_reg')
first_results

,MAE,MSE,RMSE,MAPE,R2,adj_r2
lin_reg,4.770462,31.919501,5.649735,4.474087e+13,0.429553,0.334618


In [37]:
preds = lr.predict(X_test_proc)

In [38]:
preds = pd.DataFrame(preds)

In [39]:
preds.describe()

,0
count,793.000000
mean,10.517135
std,4.093834
min,1.244186
25%,7.264032
50%,9.562714
75%,14.711824
max,18.195340


In [40]:
from sklearn.tree import DecisionTreeRegressor

dt = DecisionTreeRegressor()

In [41]:
dt.fit(X_train_proc, y_train)

DecisionTreeRegressor()

In [42]:
dt_results = evaluate_regression(dt, X_test_proc, y_test, 'dt_no_tune')

In [43]:
dt_results

,MAE,MSE,RMSE,MAPE,R2,adj_r2
dt_no_tune,5.70744,64.214376,8.013387,1.022255e+14,-0.147603,-0.338589


In [45]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(criterion='squared_error')

In [46]:
rf.fit(X_train_proc, y_train)

RandomForestRegressor()

In [47]:
rf_try = evaluate_regression(rf, X_test_proc, y_test, 'forest')
rf_try

,MAE,MSE,RMSE,MAPE,R2,adj_r2
forest,4.310668,29.554463,5.436402,3.555174e+13,0.471819,0.383919
